In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jun 29 23:59:47 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('d5_avg_error_adjustedloads.csv')
df = df.astype({col: 'float32' for col in df.select_dtypes('int64').columns})
#df = df.drop(df.filter(regex='voltage').columns, axis=1)
df['id']=np.divmod(np.arange(len(df)),2)[0]+1
df

,category,error_1,error_2,error_3,error_4,error_5,error_6,error_7,error_8,load_1_power,load_2_power,load_3_power,load_4_power,load_5_power,load_6_power,load_7_power,load_8_power,load_9_power,load_10_power,load_11_power,load_12_power,load_13_power,load_14_power,load_15_power,load_16_power,load_17_power,load_18_power,load_19_power,load_20_power,load_21_power,load_22_power,load_23_power,load_24_power,load_25_power,load_26_power,load_27_power,load_28_power,load_29_power,load_30_power,load_31_power,load_32_power,load_33_power,load_34_power,load_35_power,load_36_power,load_37_power,load_38_power,load_39_power,load_40_power,load_41_power,load_42_power,load_43_power,load_44_power,load_45_power,load_46_power,load_47_power,load_48_power,load_49_power,load_50_power,load_51_power,load_52_power,load_53_power,load_54_power,load_55_power,label,id
0,5a,0.071300,0.000896,0.000820,0.008807,0.004231,0.001658,0.001584,0.000484,1.177471,0.197054,0.604488,1.256384,0.579067,0.840147,1.382240,0.918765,1.187285,1.369714,0.936143,0.331095,1.085148,0.579615,1.553391,1.023118,0.379017,0.633642,0.323057,0.463828,0.232460,0.172173,1.077655,0.321732,0.110715,0.870895,0.686538,0.971928,0.314929,0.660298,0.705587,0.470853,1.090922,1.346202,0.474868,0.678354,1.046296,0.516885,0.147477,0.666236,0.551769,0.584022,0.041885,0.469102,1.774181,0.686337,0.791973,1.155089,0.481334,0.250535,0.330235,0.390994,0.127686,0.237726,0.388191,0.0,1
1,5a,0.200103,0.000896,0.000820,0.008807,0.004231,0.001658,0.001584,0.000484,0.571820,0.197054,0.604488,1.256384,0.579067,0.840147,1.382240,0.918765,1.187285,1.369714,0.936143,0.331095,1.085148,0.579615,1.553391,1.023118,0.379017,0.633642,0.323057,0.463828,0.232460,0.172173,1.077655,0.321732,0.110715,0.870895,0.686538,0.971928,0.314929,0.660298,0.705587,0.470853,1.090922,1.346202,0.474868,0.678354,1.046296,0.516885,0.147477,0.666236,0.551769,0.584022,0.041885,0.469102,1.774181,0.686337,0.791973,1.155089,0.481334,0.250535,0.330235,0.390994,0.127686,0.237726,0.388191,1.0,1
2,5a,0.068615,0.001102,0.000587,0.006721,0.004625,0.001498,0.001635,0.000556,1.229246,0.210630,0.800589,0.638934,0.443519,0.831630,1.181975,0.858599,0.948085,2.228018,0.949462,0.444440,0.871808,0.668130,0.854483,1.147956,0.367982,0.878881,0.339945,0.395298,0.361748,0.193831,0.957013,0.363664,-0.234085,0.800098,0.563146,0.945711,0.229572,0.433596,0.830866,0.711384,1.045827,0.834541,0.434173,0.518295,1.000345,0.473434,0.854214,0.410525,0.543098,0.568072,0.103455,0.406818,1.272575,1.251353,0.701307,1.194448,0.415428,0.288665,0.350211,0.497445,0.158514,0.253902,0.314560,0.0,2
3,5a,0.212560,0.001102,0.000587,0.006721,0.004625,0.001498,0.001635,0.000556,0.596963,0.210630,0.800589,0.638934,0.443519,0.831630,1.181975,0.858599,0.948085,2.228018,0.949462,0.444440,0.871808,0.668130,0.854483,1.147956,0.367982,0.878881,0.339945,0.395298,0.361748,0.193831,0.957013,0.363664,-0.234085,0.800098,0.563146,0.945711,0.229572,0.433596,0.830866,0.711384,1.045827,0.834541,0.434173,0.518295,1.000345,0.473434,0.854214,0.410525,0.543098,0.568072,0.103455,0.406818,1.272575,1.251353,0.701307,1.194448,0.415428,0.288665,0.350211,0.497445,0.158514,0.253902,0.314560,1.0,2
4,5a,0.063695,0.000996,0.000662,0.007110,0.004600,0.001569,0.001647,0.000585,1.181263,0.375339,0.792686,0.483117,0.563617,0.732629,1.188650,0.706094,0.934342,1.175878,1.066776,1.353485,0.947842,0.632963,0.938682,0.965707,0.544361,0.627674,0.341504,0.392944,0.311017,0.174074,0.838146,0.371818,-0.103060,0.711052,0.373761,0.919465,0.287337,0.311667,0.887670,0.452871,0.983020,0.844519,0.525112,0.624389,0.902815,0.342620,0.053365,0.420117,0.592067,0.522726,-0.038274,0.442857,1.364579,1.144914,0.715857,1.094788,0.314482,0.328803,0.357694,0.445921,0.179345,0.344106,0.357403,0.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
import numpy as np

#train, test, y_train, y_test = train_test_split(df, df['label'], test_size=0.20, stratify=df['label'])
#x_train = train.drop(columns=['label', 'category'])
#x_test = test.drop(columns=['label', 'category'])

train_inds, test_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(df, groups=df['id']))
x_train = df.iloc[train_inds].drop(columns=['label', 'category', 'id'])
x_test = df.iloc[test_inds].drop(columns=['label', 'category', 'id'])
y_train = df.iloc[train_inds]['label']
y_test = df.iloc[test_inds]['label']

In [ ]:
x_train

,error_1,error_2,error_3,error_4,error_5,error_6,error_7,error_8,load_1_power,load_2_power,load_3_power,load_4_power,load_5_power,load_6_power,load_7_power,load_8_power,load_9_power,load_10_power,load_11_power,load_12_power,load_13_power,load_14_power,load_15_power,load_16_power,load_17_power,load_18_power,load_19_power,load_20_power,load_21_power,load_22_power,load_23_power,load_24_power,load_25_power,load_26_power,load_27_power,load_28_power,load_29_power,load_30_power,load_31_power,load_32_power,load_33_power,load_34_power,load_35_power,load_36_power,load_37_power,load_38_power,load_39_power,load_40_power,load_41_power,load_42_power,load_43_power,load_44_power,load_45_power,load_46_power,load_47_power,load_48_power,load_49_power,load_50_power,load_51_power,load_52_power,load_53_power,load_54_power,load_55_power
0,0.071300,0.000896,0.000820,0.008807,0.004231,0.001658,0.001584,0.000484,1.177471,0.197054,0.604488,1.256384,0.579067,0.840147,1.382240,0.918765,1.187285,1.369714,0.936143,0.331095,1.085148,0.579615,1.553391,1.023118,0.379017,0.633642,0.323057,0.463828,0.232460,0.172173,1.077655,0.321732,0.110715,0.870895,0.686538,0.971928,0.314929,0.660298,0.705587,0.470853,1.090922,1.346202,0.474868,0.678354,1.046296,0.516885,0.147477,0.666236,0.551769,0.584022,0.041885,0.469102,1.774181,0.686337,0.791973,1.155089,0.481334,0.250535,0.330235,0.390994,0.127686,0.237726,0.388191
1,0.200103,0.000896,0.000820,0.008807,0.004231,0.001658,0.001584,0.000484,0.571820,0.197054,0.604488,1.256384,0.579067,0.840147,1.382240,0.918765,1.187285,1.369714,0.936143,0.331095,1.085148,0.579615,1.553391,1.023118,0.379017,0.633642,0.323057,0.463828,0.232460,0.172173,1.077655,0.321732,0.110715,0.870895,0.686538,0.971928,0.314929,0.660298,0.705587,0.470853,1.090922,1.346202,0.474868,0.678354,1.046296,0.516885,0.147477,0.666236,0.551769,0.584022,0.041885,0.469102,1.774181,0.686337,0.791973,1.155089,0.481334,0.250535,0.330235,0.390994,0.127686,0.237726,0.388191
2,0.068615,0.001102,0.000587,0.006721,0.004625,0.001498,0.001635,0.000556,1.229246,0.210630,0.800589,0.638934,0.443519,0.831630,1.181975,0.858599,0.948085,2.228018,0.949462,0.444440,0.871808,0.668130,0.854483,1.147956,0.367982,0.878881,0.339945,0.395298,0.361748,0.193831,0.957013,0.363664,-0.234085,0.800098,0.563146,0.945711,0.229572,0.433596,0.830866,0.711384,1.045827,0.834541,0.434173,0.518295,1.000345,0.473434,0.854214,0.410525,0.543098,0.568072,0.103455,0.406818,1.272575,1.251353,0.701307,1.194448,0.415428,0.288665,0.350211,0.497445,0.158514,0.253902,0.314560
3,0.212560,0.001102,0.000587,0.006721,0.004625,0.001498,0.001635,0.000556,0.596963,0.210630,0.800589,0.638934,0.443519,0.831630,1.181975,0.858599,0.948085,2.228018,0.949462,0.444440,0.871808,0.668130,0.854483,1.147956,0.367982,0.878881,0.339945,0.395298,0.361748,0.193831,0.957013,0.363664,-0.234085,0.800098,0.563146,0.945711,0.229572,0.433596,0.830866,0.711384,1.045827,0.834541,0.434173,0.518295,1.000345,0.473434,0.854214,0.410525,0.543098,0.568072,0.103455,0.406818,1.272575,1.251353,0.701307,1.194448,0.415428,0.288665,0.350211,0.497445,0.158514,0.253902,0.314560
4,0.063695,0.000996,0.000662,0.007110,0.004600,0.001569,0.001647,0.000585,1.181263,0.375339,0.792686,0.483117,0.563617,0.732629,1.188650,0.706094,0.934342,1.175878,1.066776,1.353485,0.947842,0.632963,0.938682,0.965707,0.544361,0.627674,0.341504,0.392944,0.311017,0.174074,0.838146,0.371818,-0.103060,0.711052,0.373761,0.919465,0.287337,0.311667,0.887670,0.452871,0.983020,0.844519,0.525112,0.624389,0.902815,0.342620,0.053365,0.420117,0.592067,0.522726,-0.038274,0.442857,1.364579,1.144914,0.715857,1.094788,0.314482,0.328803,0.357694,0.445921,0.179345,0.344106,0.357403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1535,0.172365,0.001085,0.001067,0.009693,0.010529,0.001898,0.005378,0

In [ ]:
y_train

0       0.0
1       1.0
2       0.0
3       1.0
4       0.0
       ... 
1535    1.0
1536    0.0
1537    1.0
1538    0.0
1539    1.0
Name: label, Length: 1232, dtype: float32

In [ ]:
x_test

,error_1,error_2,error_3,error_4,error_5,error_6,error_7,error_8,load_1_power,load_2_power,load_3_power,load_4_power,load_5_power,load_6_power,load_7_power,load_8_power,load_9_power,load_10_power,load_11_power,load_12_power,load_13_power,load_14_power,load_15_power,load_16_power,load_17_power,load_18_power,load_19_power,load_20_power,load_21_power,load_22_power,load_23_power,load_24_power,load_25_power,load_26_power,load_27_power,load_28_power,load_29_power,load_30_power,load_31_power,load_32_power,load_33_power,load_34_power,load_35_power,load_36_power,load_37_power,load_38_power,load_39_power,load_40_power,load_41_power,load_42_power,load_43_power,load_44_power,load_45_power,load_46_power,load_47_power,load_48_power,load_49_power,load_50_power,load_51_power,load_52_power,load_53_power,load_54_power,load_55_power
6,0.055442,0.001211,0.001009,0.007371,0.005715,0.001538,0.002007,0.000509,1.651844,1.780131,1.114402,0.511536,0.480941,0.717667,1.128351,0.692570,0.973808,1.894256,0.895196,1.563610,1.821401,0.619964,1.161224,1.023949,0.458036,0.827084,0.306680,0.424759,0.402088,0.177467,0.710330,0.367398,-0.258117,0.715840,0.476163,0.900317,0.211737,0.422923,0.926358,0.500431,1.043072,1.035985,0.405935,0.575969,0.890533,0.488084,-0.037888,0.473438,0.731123,0.479099,0.042422,0.357853,1.385116,2.045551,0.704819,1.250473,0.397851,0.235754,0.306498,0.465465,0.127248,0.238090,0.322690
7,0.180719,0.001211,0.001009,0.007371,0.005715,0.001538,0.002007,0.000509,0.802191,1.780131,1.114402,0.511536,0.480941,0.717667,1.128351,0.692570,0.973808,1.894256,0.895196,1.563610,1.821401,0.619964,1.161224,1.023949,0.458036,0.827084,0.306680,0.424759,0.402088,0.177467,0.710330,0.367398,-0.258117,0.715840,0.476163,0.900317,0.211737,0.422923,0.926358,0.500431,1.043072,1.035985,0.405935,0.575969,0.890533,0.488084,-0.037888,0.473438,0.731123,0.479099,0.042422,0.357853,1.385116,2.045551,0.704819,1.250473,0.397851,0.235754,0.306498,0.465465,0.127248,0.238090,0.322690
24,0.119808,0.000843,0.000751,0.013383,0.005553,0.003747,0.001642,0.000763,0.485547,0.422467,0.231022,0.575706,0.761860,0.484564,0.675419,0.743290,1.603946,0.533670,1.253402,0.513468,0.774047,1.078789,0.027211,0.415903,0.582808,0.499861,0.698217,0.840388,0.225640,1.598576,0.259028,0.311478,0.906669,1.522483,0.917065,0.927534,0.605878,0.555903,0.556051,0.118070,0.367139,1.613427,0.849279,0.926524,3.347192,0.397445,1.860061,0.545848,0.420524,1.166453,0.337348,0.443245,0.328373,1.131374,1.233874,0.326351,0.318980,0.558127,0.775663,0.357359,0.356793,0.311581,0.735449
25,0.195359,0.000843,0.000751,0.013383,0.005553,0.003747,0.001642,0.000763,0.485547,0.177956,0.231022,0.575706,0.761860,0.484564,0.675419,0.743290,1.603946,0.533670,1.253402,0.513468,0.774047,1.078789,0.027211,0.415903,0.582808,0.499861,0.698217,0.840388,0.225640,1.598576,0.259028,0.311478,0.906669,1.522483,0.917065,0.927534,0.605878,0.555903,0.556051,0.118070,0.367139,1.613427,0.849279,0.926524,3.347192,0.397445,1.860061,0.545848,0.420524,1.166453,0.337348,0.443245,0.328373,1.131374,1.233874,0.326351,0.318980,0.558127,0.775663,0.357359,0.356793,0.311581,0.735449
34,0.113131,0.001091,0.001023,0.014136,0.006621,0.002706,0.002246,0.001355,0.242064,0.595705,0.558725,1.401018,0.238815,1.147390,1.291566,0.712222,0.660460,0.174784,1.923451,1.190487,1.421874,1.103513,2.101953,0.682847,0.793778,0.459602,1.247732,0.418861,0.479765,0.652119,0.371805,0.439062,0.800292,0.264731,0.383682,0.605705,0.939015,1.489306,0.449216,1.154785,0.045788,1.328412,0.378983,0.485347,1.776730,0.488761,0.288048,0.267331,0.361207,1.049244,0.266688,2.011802,0.817719,0.440642,0.980519,1.946709,0.344645,0.852372,0.468914,0.629896,0.675858,1.060297,0.686088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1517,0.103960,0.000641,-0.002478,0.011454,0.006699,0.002081,0.0257

In [ ]:
y_test

6       0.0
7       1.0
24      0.0
25      1.0
34      0.0
       ... 
1517    1.0
1520    0.0
1521    1.0
1522    0.0
1523    1.0
Name: label, Length: 308, dtype: float32

In [ ]:
# Scale the train set
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train_scaled = sc.fit_transform(x_train)
x_test_scaled = sc.transform(x_test)

In [ ]:
d1 = pd.read_csv('d1_avg_error_adjustedloads.csv')
d1['id']=np.divmod(np.arange(len(d1)),2)[0]+1
train1_inds, test1_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(d1, groups=d1['id']))
x1_test = d1.iloc[test1_inds].drop(columns=['label', 'category', 'id'])
y1_test = d1.iloc[test1_inds]['label']

d2 = pd.read_csv('d2_avg_error_adjustedloads.csv')
d2['id']=np.divmod(np.arange(len(d2)),2)[0]+1
train2_inds, test2_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(d2, groups=d2['id']))
x2_test = d2.iloc[test2_inds].drop(columns=['label', 'category', 'id'])
y2_test = d2.iloc[test2_inds]['label']

d3 = pd.read_csv('d3_avg_error_adjustedloads.csv')
d3['id']=np.divmod(np.arange(len(d3)),2)[0]+1
train3_inds, test3_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(d3, groups=d3['id']))
x3_test = d3.iloc[test3_inds].drop(columns=['label', 'category', 'id'])
y3_test = d3.iloc[test3_inds]['label']

d4 = pd.read_csv('d4_avg_error_adjustedloads.csv')
d4['id']=np.divmod(np.arange(len(d4)),2)[0]+1
train4_inds, test4_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(d4, groups=d4['id']))
x4_test = d4.iloc[test4_inds].drop(columns=['label', 'category', 'id'])
y4_test = d4.iloc[test4_inds]['label']

d5 = pd.read_csv('d5_avg_error_adjustedloads.csv')
d5['id']=np.divmod(np.arange(len(d5)),2)[0]+1
train5_inds, test5_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(d5, groups=d5['id']))
x5_test = d5.iloc[test5_inds].drop(columns=['label', 'category', 'id'])
y5_test = d5.iloc[test5_inds]['label']

d6 = pd.read_csv('d6_avg_error_adjustedloads.csv')
d6['id']=np.divmod(np.arange(len(d6)),2)[0]+1
train6_inds, test6_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(d6, groups=d6['id']))
x6_test = d6.iloc[test6_inds].drop(columns=['label', 'category', 'id'])
y6_test = d6.iloc[test6_inds]['label']

d7 = pd.read_csv('d7_avg_error_adjustedloads.csv')
d7['id']=np.divmod(np.arange(len(d7)),2)[0]+1
train7_inds, test7_inds = next(GroupShuffleSplit(test_size=0.20, n_splits=2).split(d7, groups=d7['id']))
x7_test = d7.iloc[test7_inds].drop(columns=['label', 'category', 'id'])
y7_test = d7.iloc[test7_inds]['label']


x1_test_scaled = sc.transform(x1_test)
x2_test_scaled = sc.transform(x2_test)
x3_test_scaled = sc.transform(x3_test)
x4_test_scaled = sc.transform(x4_test)
x5_test_scaled = sc.transform(x5_test)
x6_test_scaled = sc.transform(x6_test)
x7_test_scaled = sc.transform(x7_test)

x2_test

,error_1,error_2,error_3,error_4,error_5,error_6,error_7,error_8,load_1_power,load_2_power,load_3_power,load_4_power,load_5_power,load_6_power,load_7_power,load_8_power,load_9_power,load_10_power,load_11_power,load_12_power,load_13_power,load_14_power,load_15_power,load_16_power,load_17_power,load_18_power,load_19_power,load_20_power,load_21_power,load_22_power,load_23_power,load_24_power,load_25_power,load_26_power,load_27_power,load_28_power,load_29_power,load_30_power,load_31_power,load_32_power,load_33_power,load_34_power,load_35_power,load_36_power,load_37_power,load_38_power,load_39_power,load_40_power,load_41_power,load_42_power,load_43_power,load_44_power,load_45_power,load_46_power,load_47_power,load_48_power,load_49_power,load_50_power,load_51_power,load_52_power,load_53_power,load_54_power,load_55_power
0,0.128482,0.000553,0.000697,0.014280,0.006118,0.002103,0.002327,0.001358,0.428484,0.605475,0.122023,0.500988,0.503125,0.058839,0.982732,1.203046,0.318667,0.329418,0.523286,0.388265,0.414652,0.712255,1.731068,0.310328,1.107882,0.103660,0.160542,0.317223,0.572434,0.431195,0.425236,0.998350,0.660810,0.016832,0.437352,0.427024,0.446572,0.477836,0.291881,0.606122,0.496125,0.555117,1.411677,1.016161,0.510746,1.140802,0.792257,0.468473,0.542465,0.502236,0.485455,1.574415,1.016350,0.669295,0.972781,0.864103,0.714378,0.991250,1.062161,0.696616,0.904205,0.288144,1.288197
1,0.199792,0.000553,0.000697,0.014280,0.006118,0.002103,0.002327,0.001358,0.251133,0.605475,0.122023,0.500988,0.503125,0.058839,0.982732,1.203046,0.318667,0.329418,0.523286,0.388265,0.414652,0.712255,1.731068,0.310328,1.107882,0.103660,0.160542,0.317223,0.572434,0.431195,0.425236,0.998350,0.660810,0.016832,0.437352,0.427024,0.446572,0.477836,0.291881,0.606122,0.496125,0.555117,1.411677,1.016161,0.510746,1.140802,0.792257,0.468473,0.542465,0.502236,0.485455,1.574415,1.016350,0.669295,0.972781,0.864103,0.714378,0.991250,1.062161,0.696616,0.904205,0.288144,1.288197
10,0.156115,0.000410,0.000683,0.024348,0.008586,0.003120,0.003535,0.001915,0.372969,0.606789,0.194617,1.393885,0.311667,0.027404,0.837524,0.954983,0.356781,0.311035,0.358132,0.319713,0.442468,0.930980,0.226721,0.422740,1.676706,0.229150,0.338032,0.330853,0.565992,0.500084,0.546712,1.119868,0.497112,0.879707,0.556125,0.366133,0.849033,0.417325,1.653644,0.690160,0.617162,0.648278,1.594722,1.339606,0.481852,1.510646,0.843608,0.551983,0.417949,0.537100,0.355290,1.337477,1.310706,0.798883,1.408148,0.988300,0.688615,0.956334,1.152207,0.664977,1.788341,0.875507,1.851027
11,0.210043,0.000410,0.000683,0.024348,0.008586,0.003120,0.003535,0.001915,0.218596,0.606789,0.194617,1.393885,0.311667,0.027404,0.837524,0.954983,0.356781,0.311035,0.358132,0.319713,0.442468,0.930980,0.226721,0.422740,1.676706,0.229150,0.338032,0.330853,0.565992,0.500084,0.546712,1.119868,0.497112,0.879707,0.556125,0.366133,0.849033,0.417325,1.653644,0.690160,0.617162,0.648278,1.594722,1.339606,0.481852,1.510646,0.843608,0.551983,0.417949,0.537100,0.355290,1.337477,1.310706,0.798883,1.408148,0.988300,0.688615,0.956334,1.152207,0.664977,1.788341,0.875507,1.851027
12,0.103762,0.000475,0.000615,0.014381,0.006302,0.002111,0.002942,0.001692,0.738833,0.774431,0.124387,1.704978,0.584297,0.023901,0.819714,0.984054,0.448483,0.497939,0.340265,0.305583,0.545456,0.848416,0.127867,0.304623,1.578242,0.215132,0.293967,0.492722,0.521621,0.388937,0.519616,0.813418,0.496589,0.674390,0.591559,0.433837,0.463706,0.406033,0.761852,0.684567,0.442947,0.592435,1.463909,1.004060,0.329193,1.377542,0.842921,0.354229,0.435658,0.486184,0.488760,1.762592,0.725076,0.659919,1.776861,0.932605,0.777828,0.830223,0.999419,0.817367,1.979411,0.163590,0.940585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1499,0.060946,0.000759,0.000538,0.008062,0.005146,0.001782,0.001814,0.

In [ ]:
# Install thundersvm stuff
#!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
#!ls  # Check if required cuda 9.0 amd64-deb file is downloaded
#!dpkg -i cuda-repo-ubuntu1704-9-0-local_9.0.176-1_amd64-deb
#!ls /var/cuda-repo-9-0-local | grep .pub
#!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
#!apt-get update
#!sudo apt-get install cuda-9.0
#!nvcc --version
#!pip install thundersvm

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from thundersvm import SVC

parameters = [{'C': [2**x for x in range(-15,4) if x%2 != 0], 'kernel': ['rbf'], 'gamma': [2**x for x in range(-15,4) if x%2 != 0]}]
parameters2 = [{'C': [8], 'kernel': ['rbf'], 'gamma': [0.0078125]}]
#parameters = [{'C': [2**x for x in range(-15,4) if x%2 != 0], 'kernel': ['rbf'], 'gamma': [0.5]}]
grid_search = GridSearchCV(estimator = SVC(),
                           param_grid = parameters2,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = 1, 
                           verbose = 4)

# Fit the scaled training data to grid-search
grid_search = grid_search.fit(x_train_scaled, y_train)
accuracy = grid_search.best_score_
accuracy

Fitting 10 folds for each of 1 candidates, totalling 10 fits
[CV] C=8, gamma=0.0078125, kernel=rbf ................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.694, total=   0.5s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.927, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.854, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.821, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.967, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s


[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.642, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.715, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.642, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.724, total=   0.0s
[CV] C=8, gamma=0.0078125, kernel=rbf ................................
[CV] .... C=8, gamma=0.0078125, kernel=rbf, score=0.837, total=   0.0s


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.9s finished


0.782421977445581

In [ ]:
grid_search.best_params_

{'C': 8, 'gamma': 0.0078125, 'kernel': 'rbf'}

In [ ]:
grid_search.best_score_

0.782421977445581

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

# Predict the test data
#y_pred = grid_search.predict(x1_test_scaled) 

# Compare predicted to Y_test
#tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
#acc = (tp + tn) / (tn + fp + fn + tp)
#pr = tp / (tp + fp)
#dr = tp / (tp + fn)
#f1 = (2*dr*pr) / (dr + pr)

#x_test_list = [x1_test_scaled, x2_test_scaled, x3_test_scaled, x4_test_scaled, x_test_scaled, x6_test_scaled, x_test_scaled]
#y_test_list = [y1_test, y2_test, y3_test, y4_test, y_test, y6_test, y7_test]

#acc_list = []
#pr_list = []
#dr_list = []
#f1_list = []
  
y_pred = grid_search.predict(x_test_scaled) 
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
acc = (tp + tn) / (tn + fp + fn + tp)
pr = tp / (tp + fp)
dr = tp / (tp + fn)
f1 = (2*dr*pr) / (dr + pr)


In [ ]:
print('True positives: {}'.format(tp))
print('True negatives: {}'.format(tn))
print('False positives: {}'.format(fp))
print('False negatives: {}'.format(fn))

True positives: 109
True negatives: 147
False positives: 7
False negatives: 45


In [ ]:
acc

0.8311688311688312

In [ ]:
pr

0.9396551724137931

In [ ]:
dr

0.7077922077922078

In [ ]:
f1

0.8074074074074074

In [ ]:
#from sklearn.metrics import classification_report
#print(classification_report(y_test, y_pred))